In [3]:
from ultralytics import YOLO

def main():
    # 1. Load model xịn nhất của bạn
    model = YOLO(r'D:\ADMIN\Documents\Classwork\CV\runs\detect\sroie_best_v2\weights\best.pt')

    # 2. Chạy đánh giá trên tập Test
    print("--- BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST ---")
    metrics = model.val(
        data=r'D:\ADMIN\Documents\Classwork\CV\Datasets\SROIE_YOLO_TRAIN\data.yaml',
        split='test',   # QUAN TRỌNG: Chỉ định dùng tập test
        imgsz=1024,     # Phải dùng cùng độ phân giải lúc train v2
        batch=8,
        conf=0.001,     # Ngưỡng thấp để tính mAP chính xác nhất
        iou=0.6,        # Ngưỡng IoU khi test
        name='eval_test_set'
    )

    # 3. In kết quả tóm tắt
    print(f"\nKẾT QUẢ TEST SET:")
    print(f"mAP50:    {metrics.box.map50:.3f}")
    print(f"mAP50-95: {metrics.box.map:.3f}")
    print(f"Precision: {metrics.box.mp:.3f}")
    print(f"Recall:    {metrics.box.mr:.3f}")

if __name__ == '__main__':
    main()

--- BẮT ĐẦU ĐÁNH GIÁ TRÊN TẬP TEST ---
Ultralytics 8.3.232  Python-3.11.13 torch-2.8.0+cpu CPU (13th Gen Intel Core i5-1340P)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 92.691.6 MB/s, size: 800.5 KB)
val: Scanning D:\ADMIN\Documents\Classwork\CV\Datasets\SROIE_YOLO_TRAIN\test\labels... 347 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 347/347 367.4it/s 0.9s0.1s
val: D:\ADMIN\Documents\Classwork\CV\Datasets\SROIE_YOLO_TRAIN\test\images\X51005230659.jpg: 1 duplicate labels removed
val: New cache created: D:\ADMIN\Documents\Classwork\CV\Datasets\SROIE_YOLO_TRAIN\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 44/44 4.9s/it 3:343.4ss
                   all        347      18704      0.974      0.966      0.981      0.723
Speed: 1.1ms preprocess, 578.3ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to D